*A quick note: You can run all of the code blocks below by hovering your mouse over the brackets at the top left of each block and clicking the play button that appears. In order to run a block of code, you have to have already run the subsequent blocks (In other words you have to run the code blocks in order)*

# **Analyzing Data About Bank Customers Using My Own Neural Network Engine**

In this Python notebook, I will be showcasing a more advanced and applicable use of the neural network engine that I created from scratch. This Python notebook is an adaptation of two Python notebooks that I have created in the past. I would suggest reading both of the following notebooks before continuing with the following:

* [Bank Data Analysis Deep Learning](https://colab.research.google.com/drive/1stLWnNExen8nat0gEdQbnEgCeZ3UdGnO?usp=sharing)

* [Neural Network Engine From Scratch](https://colab.research.google.com/drive/1lOPb-V4UYwZnURc1HNYiKn4cuzKdpRTx?usp=sharing)

I will be using the neural network engine I created and explained in the second notebook which functions similarly to libraries like TensorFlow/Keras or PyTorch in order to solve the machine learning problem from the first notebook linked above.

**Run the code below to load the neural network engine that is fully explained in the notebook linked above.**

In [ ]:
from random import random
#In this case we will need the exponential function from numpy because the 
#default Python exponential function can't handle the numbers in this case.
import numpy as np

class NeuralNetwork(object):
    """
    This class will create an instance of a neural network. The idea is that we 
    will be able to use the methods in this class to customize the neural network
    to the exact size that we want it to be (including the number of layers, number
    of neurons per layer, and the activations of the neurons). This means that we
    will need our forward and backpropgation algorithms to be programmed so that
    they will function properly no matter the size of the network.

    The code for this class is split into four main steps:
    1. Initialization: In this section, we enable the creation and customization 
       of the neural network (adding layers, changing activations, etc.)
    2. Forward Propagation: In this section, we create the forward propagation
       algorithm which will both store the outputs of each neuron during forward
       propagation and will return the output of the network (thus it can be used)
       when our network is fully trained to make predictions.
    3. Backward Propagation: In this section, we create the backward propagation
       algorithm which calculates the errors of each neuron in the network based
       on the error of the network calculated after forward propagation and adjusts
       the weights of each neuron accordingly.
    4. Training: In this section, we create the function which will enable us to
       train our network with specific training data. This is where we will be able 
       to specify relevant learning parameters like the learning rate and number of
       epochs that we want.
    """

    # Section 1: Initialization 
    def __init__(self):
        """
        When we first initialize the network, we don't need to do much. All we do
        here is create an empty list for our neural network (which can be filled)
        in later.
        """
        self.neural_network = list()
    
    def add_layer(self, num_neurons, activation = 'sigmoid', input_layer = False):
        """
        This is the function that allows the user to add layers to the neural 
        network. They can specify the number of neurons in the layer, the activation
        function used by the neurons in the layer (which is set by default to the
        sigmoid function but can be adjusted), and if the layer is the input layer.
        Note that the first layer added to the network must always be specified 
        as the input layer.
        """
        if not input_layer:
            #Each layer is represented as a list of neurons where each neuron
            #is represented as a dictionary with a bias, a set of weights corresponding
            #to the neurons in the previous layer, and an activation function.
            self.neural_network.append([{'bias': random(), 
                                         'weights': [random() for i in range(self.last_layer_neurons)],
                                         'activation': activation} 
                                         for i in range(num_neurons)])
        self.last_layer_neurons = num_neurons

    # Section 2: Forward Propagation
    def sigmoid(self, value, derivative = False):
        """
        This is the Sigmoid activation function which can be used in computing the
        activation of a neuron. It can be also used as the derivative of the sigmoid
        function (which you will see we need later in backpropagation).
        """
        if derivative:
            return value * (1 - value)
        return 1/(1 + np.exp(-1 * value))
    
    def relu(self, value, derivative = False):
        """
        This is the Rectified Linear Unit (ReLU) activation function which can be 
        used in computing the activation of a neuron. This function returns a value 
        of 0 if it is fed a value less than 0 and returns the value it was fed if 
        the value is greater than 0. It can also be used as the derivative of the 
        relu function (which again we will need in backpropagation).
        """
        if derivative:
            return 1 if value > 0 else 0
        return value if value > 0 else 0
    
    def activate(self, inputs, weights, bias, activation):
        """
        This function computes the activation of a specific neuron. It computes
        the dot product of the input and weight vectors (basically just multiplying
        all of the weights coming into the neuron with the corresponding outputs
        of the neurons they are coming from and then adding all of them up) and 
        then adds the bias to this number before passing the whole sum into the
        specified activation function.
        """
        activation_functions = {'sigmoid': self.sigmoid, 'relu': self.relu}
        activation_function = activation_functions[activation]
        
        return activation_function(sum([inputs[i] * weights[i] for i in range(len(inputs))]) + bias)
    
    def forward_propagate(self, inputs):
        """
        This function passes the specified inputs into the input layer of the neural
        network. From there, it computes the activation of each neuron in a layer 
        using the fuction above and then passes these activations to the neurons
        of the next layer, repeating this process until it reaches the output layer.
        """
        for layer in self.neural_network:
            layer_outputs = list()
            for neuron in layer:
                #Here we store the outputs of each neuron (computed using the activate
                #function we defined above) because we will need them for backpropagation
                neuron['output'] = self.activate(inputs, neuron['weights'], neuron['bias'], neuron['activation'])
                layer_outputs.append(neuron['output'])
            #Here is where we pass the outputs of one layer as the inputs of the next layer
            inputs = layer_outputs
        #Here we return the output of the final layer (which is the prediction of the network)
        return inputs
    
    # Section 3: Backward Propagation
    def backward_propagate(self, inputs, expected_outputs, learning_rate):
        """
        This function calculates the error of our network (the difference between
        the output predicted by the network and the output it was supposed to predict).
        In then uses this error of the last neurons in the network to calculate the 
        error of the neurons in the previous layer using some simple calculus. 
        Then, it adjust the weights and biases of our neural network based on the
        errors of each neuron (this is what actually makes the network more accurate).
        """
        #Here, we compute the errors of each neuron, focusing on the layers
        #In reverse order where we start by looking at the last layer
        for i in reversed(range(len(self.neural_network))):
            layer = self.neural_network[i]
            layer_errors = list()
            #If we are looking at the last layer...
            if i == len(self.neural_network) - 1:
                for j in range(len(layer)):
                    neuron = layer[j]
                    #Compute the error of the output neurons in our neural network
                    neuron_error = expected_outputs[j] - neuron['output']
                    layer_errors.append(neuron_error)
            else:
                #For any other layer besides the last layer...
                for j in range(len(layer)):
                    #Calculate the error associated with each neuron based on the 
                    #connected neurons in the following layer
                    neuron_error = sum([neuron['weights'][j] * neuron['error'] for neuron in self.neural_network[i + 1]])
                    layer_errors.append(neuron_error)
            for j in range(len(layer)):
                neuron = layer[j]
                #Finally, multiply the errors we found before by the derivative of 
                #activation function of each neuron (this comes from using the chain
                #rule from calculus).
                neuron['error'] = layer_errors[j] * self.sigmoid(neuron['output'], True)
        
        #Here, we adjust the weights of our network based on the errors we computed
        #This time we go through the neural network in order from the first to last layer
        for i in range(len(self.neural_network)):
            layer = self.neural_network[i]
            for neuron in layer:
                for j in range(len(neuron['weights'])):
                    #Here we adjust the weights based on the error associated with
                    #each neuron and the outputs of the previous layer. We also
                    #take into account the learning rate to specify how much we want
                    #to adjust our weights for each training example.
                    neuron['weights'][j] += learning_rate * inputs[j] * neuron['error']
                #Here we adjust the bias taking into account the learning rate
                neuron['bias'] += learning_rate * neuron['error']
            #Here we save the outputs of the current layer so that we can reference them
            #When adjusting the weights of the next layer.
            inputs = [neuron['output'] for neuron in self.neural_network[i]]
    
    #Section 4: Training
    def train(self, x_train, y_train, learning_rate, epochs):
        """
        Finally, we create the method which we wil use to access all of the above
        methods. This method will allow us to pass in training data into our neural
        network. The network will train with the specified number of epochs (iterations
        through the training data) and will use the specified learning rate.
        
        """
        for epoch in range(epochs):
            total_error = 0
            for i in range(len(x_train)):
                x = x_train[i]
                y = y_train[i]
                #Propagate our training inputs through our network
                outputs = self.forward_propagate(x)
                #Calculate the mean squared error of the predicted outputs (for display purposes)
                total_error += sum([(y[j] - outputs[j]) ** 2 for j in range(len(outputs))])
                #Adjust weights and biases with backpropagation
                self.backward_propagate(x, y, learning_rate)
            #Print a message to the console at the end of each epoch so we can see
            #The progress of the neural network/if it is improving
            print("Epoch: {}, Total Error: {:.3f}".format(epoch + 1, total_error))

**Making a Prediction Function**

This function allows us to assess the accuracy of our model once it is already trained. It tests the models predictions for specific values against what the actual values are and returns the number of correct and incorrect answers that the model provided. This will serve as a way for us to adress if our model is working properly later on.

In [ ]:
def predict(model, x_test, y_test):
    num_correct = 0
    num_wrong = 0
    for i in range(len(x_test)):
        x = x_test[i]
        y = y_test[i]
        outputs = model.forward_propagate(x)
        outputs = 1 if outputs[0] > .5 else 0 
        if outputs == y[0]:
            num_correct += 1
        else:
            num_wrong += 1
    print("Correct: {}, Incorrect: {}, Accuracy: {}".format(num_correct, num_wrong, num_correct/(num_correct + num_wrong)))

# **The Dataset**
Here is the dataset we will be using: [Bank Customer Data Spreadsheet](https://drive.google.com/file/d/0B6eU_Ir83rAKOVdxV0MzazdNQlE1TXJNNDJOS2lDaFFYMkww/view?usp=sharing)

***If you want to follow along and run the code below, click on the dataset above and hit the download button in the top right.***

You can see that in the left columns, there is data about each customer. The farthest right column stores a value of 1 if the customer ended up staying with the bank after six months and 0 if the customer decided to leave.

At the end of the project, our goal is to have a program that predicts if customers will stay with a bank in the next six months based on statistics about the customer.

**Upload the Dataset to Python**


1.   Run the below code and click the "Choose Files" button that appears
2.   Select the "Bank_Customer_Data.csv" file from your file explorer

In [ ]:
from google.colab import files
uploaded = files.upload()


3.    Wait until the dataset is 100% done loading
3.    Now run the below code and the dataset is ready to go!


In [ ]:
import pandas as pd
dataset = pd.read_csv("Bank_Customer_Data.csv")

5.     If you run the following code, you can see what five rows of the dataset look like. You can adjust the numbers in the brackets to see more of the dataset. If you make the brackets look like this then you will be able to see the entire dataset: ```[:, :]```. Note that you can't see the middle columns of the table just because there are too many columns to display so they are abbreviated with elipses. The columns are still in the actual dataset though.



In [ ]:
print(dataset.iloc[1:6, :])

# **Data Preprocessing**

In this stage, we clean the data in our dataset and prepare it to be passed through our neural network. For the sake of being concise and avoiding redundancy, I won't describe every detail of what is going on here but if you want to learn more about the specifics of this code, check out my [original Python notebook](https://colab.research.google.com/drive/1stLWnNExen8nat0gEdQbnEgCeZ3UdGnO?usp=sharing) on this code where I explain it all.

**Run the code below to preprocess the dataset**

In [ ]:
# Importing numpy
import numpy as np

X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

#This code was changed because the one hot encoder API changed
label_encoder_x_1 = LabelEncoder()
X[: , 2] = label_encoder_x_1.fit_transform(X[:,2])
transformer = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), [1])], remainder ='passthrough') 

X = transformer.fit_transform(X.tolist())
X = X.astype('float64')
X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Currently all of our data is arranged into a numpy array. However, the neural network engine we build takes data in the form of standard Python arrays (which can have many dimensions). Thus, we will need to convert these numpy arrays to traditional Python arrays with the following code so that we can pass them into our custom neural network.

In [ ]:
x_train = []
for row in X_train:
    x_train.append([item for item in row])

x_test = []
for row in X_test:
    x_test.append([item for item in row])

y_train = []
for item in Y_train:
    y_train.append([item])
    
y_test = []
for item in Y_test:
    y_test.append([item])

# **Building The Neural Network**

Now that all of the data is processed properly, we can get to creating the actual neural network. We are trying to create a neural network that will take in all of the input data about the banks customers (there are 11 datapoints for each customer in the dataset) and will make a prediction as to whether they will remain with or leave the bank in the next six months. 

In [ ]:
#Create an instance of our NeuralNetwork class
model = NeuralNetwork()

#Since we have 11 datapoints about each customer, we will need an input layer with 11 inputs neurons
model.add_layer(11, input_layer = True)

#A common rule for the number of neurons in the hidden layers is to take the average of the neurons
#In the input and output layers, which in this case is 6
model.add_layer(6, activation = 'sigmoid')
model.add_layer(6, activation = 'sigmoid')

#Finally, we want to predict a single binary value so we only need one neuron in the output layer
model.add_layer(1, activation = 'sigmoid')

Now that the neural network is build, we can train the network to see how accurate it is at making predictions for this problem. When you run the code below, you will see that the neural network is training properly as the error is steadily decreasing with each new batch of training data.

In [ ]:
model.train(x_train, y_train, 0.3, 20)

Finally, we can use our model to make predictions about customers that it is has not analyzed yet (this data is coming from the testing datasets we created in the data preprocessing stage). Run the code below to see what happens.

In [ ]:
predict(model, x_test, y_test)

As you can see, the model is working very well! It usually gets to around an 85-86% accuracy level with its predictions. If you saw [my Python notebook](https://colab.research.google.com/drive/1stLWnNExen8nat0gEdQbnEgCeZ3UdGnO?usp=sharing/) where I used keras to solve this exact problem, the neural network usually got to an accuracy level of about 85%. This means that the neural network that we built completely from scratch is almost as good if not slightly better than keras, one of the industry standards for building neural networks! Not bad!

# **Thanks for Reading!**

My name is Adam Majmudar and I'm a 17 year old machine learning developer.

If you want to connect with me/see my work, check out the following links:

* [My Medium](https://medium.com/@mr.adam.maj)
* [My Linkedin](https://www.linkedin.com/in/adam-majmudar-24b596194/)
* [My Portfolio](https://tks.life/profile/adam.majmudar)